## 爬取NSB杭州工会BBS
---
**环境**：
- Python 3.5+
- requests 2.11.1
- openpyxl 2.4.8
- pandas 0.18.1
- matlibplot 2.0.0
- numpy 1.11.3+mkl

**功能**
- 爬取NSB杭州工会BBS发帖信息

**历史**
- 0.01: 2017/08/12

**TO DO**
- 增加对信息可视化分析
- 重构

In [1]:
import requests
import urllib.parse

from lxml import etree
from io import StringIO

import datetime
import time
import os
import sys
import random

import pandas as pd

#===========================================================================================
# 从wireshark中得到如下实际登陆网址，注意登陆网址不是'http://www.nsntradeunion.com.cn/bbs/mvnforum/login'
url_login = 'http://www.nsntradeunion.com.cn/bbs/mvnforum/loginprocess'
base_url = 'http://www.nsntradeunion.com.cn/bbs/mvnforum/'

# HTTP消息头
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:45.0) Gecko/20100101 Firefox/45.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'zh-CN,zh;q=0.8,en-US;q=0.6,en;q=0.4',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0',
    'Keep-Alive': 'timeout=10, max=10',
}
# HTTP登陆需要携带的数据，登陆密码经过加密，从wireshark可以获取该编码
payload = {
    'MemberName': '000000000',
    'MemberMatkhau': '',
    'md5pw': 'EcVhldcfLBOR7NJhScfvBA==',
    'url': '',
    # 保持自动登陆状态
    'AutoLogin': 'yes',
    'AutoLoginExpire': "2592000"    
}

#============================================================================================
# 获取网址内容，输入url，输出url内容
def download_url(url, headers):
    if url is None:
        return None
    # 防止爬取网站屏蔽，增加爬取时间间隔  
    time.sleep(random.randint(5,10))
    content = session.get(url, headers=headers)
    if content.status_code != 200:
        return None
    return content

# 解析网页返回element tree
def parse_url(content):
    parser = etree.HTMLParser()
    tree = etree.parse(StringIO(content.text), parser)    
    return tree

def login(url, headers=headers, data=payload):  
    # POST方法使用用户名和密码登陆，会返回index page：http://www.nsntradeunion.com.cn/bbs/mvnforum/index
    index_page = session.post(url, headers=headers, data=payload)
    if index_page.status_code != 200:
        return None    
    
    return index_page

# 判断是否登陆成功
def is_login(index_page):
    parser = etree.HTMLParser()
    tree = etree.parse(StringIO(index_page.text), parser)
    # 如果登陆成功，在如下path可以找到myprofile节点，否则说明登陆失败
    index = tree.xpath('//table/tr/td[1]/a[@href="myprofile"]')    
    if index != []:
        return True
    else:
        return False

# 获取所有板块的完整网址
def get_board_urls(index_page):
    board_urls = []
    tree = parse_url(index_page)

    # 找到Section名字
#     section_name = tree.xpath('//tr[@class="portlet-section-subheader"]/td/b/text()')
#     print('版块区名称：\n', section_name)
    # 找到所有板块子link（比如：'listthreads?forum=1'）
    board_urls = tree.xpath('//tbody/tr/td/a[contains(@class, "messageTopic")]/@href')
    
    return board_urls

def get_next_page(tree):
        # 获取当前页面下一页网址
        if (tree.xpath('//a[text()="下一页"]') != []):
            next_page_url = tree.xpath('//a[text()="下一页"]')[0].attrib['href'].strip()
        else:
            next_page_url = None  
        
        return next_page_url

# 解析每一个板块的页面，获取板块的每一个页面，并解析每一个发帖并将结果保存到post_list
def parse_board_url(next_page_url, post_data_list):
#     count = 0  # For debugging
    # 如果板块页面不是最后一页（最后一页没有下一页），则继续解析每一个页面的发帖
    while (next_page_url != None):
#         count += 1 # For debugging
        # 和base_url组成完整的link（'http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1'）    
        next_page_url = urllib.parse.urljoin(base_url, next_page_url)  
        # 获取板块页面的内容
        print("Start to parse page:", next_page_url)
        next_page_url_content = download_url(next_page_url, headers=headers)
        # 解析返回内容为element tree
        tree = parse_url(next_page_url_content)
        # 获取板块名称
        board_name = tree.xpath('//tr[contains(@class,"nav")]/td[2]/text()[3]')[0].strip().split(':')[-1]
        # 获取当前页面的所有帖子的参考节点
        post_root_list = tree.xpath('//tr[contains(@class,"portlet-section-alternate")]') 
        # 将当前页面参考节点加入到发帖列表里面并返回
        for post_root in post_root_list:
            post_data = parse_post_url(post_root, board_name)
            post_data_list.append(post_data)

        next_page_url = get_next_page(tree)  
        
#         if count>0:  # For debugging
#             return post_data_list  
    return post_data_list
        
# 解析每一个发帖内容并保存结果        
def parse_post_url(post_root, board_name):
    post_data = {}
    post_data['版块'] = board_name
    post_data['标题'] = post_root.xpath('./td[3]/a/text()')[0].strip()
    post_data['作者'] = post_root.xpath('./td[4]/a/text()')[0].strip()
    post_data['作者ID'] = post_root.xpath('./td[4]/a/@href')[0].strip().split("=")[1]
    post_data['回复数'] = post_root.xpath('./td[5]/b/text()')[0].strip()
    post_data['浏览数'] = post_root.xpath('./td[6]/b/text()')[0].strip()
    post_data['最后回复日期'] = post_root.xpath('./td[7]/text()')[0].strip()

    return post_data

#  将爬取结果保存到csv，需要pip安装openpyxl
def save_parse_data(post_data_list):
    df = pd.DataFrame(post_data_list)
    current_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    filename = 'nsnbbs' + current_time + '.csv'
    file = df.to_csv(filename)
    print("Crawled data saved in file:", filename)

In [2]:
with requests.Session() as session:
    index_page = login(url_login)
    
    if is_login(index_page):
        print("Login successfully!")
    else:
        print("Login failed, exit!")
    
    post_data_list = []
    board_urls = get_board_urls(index_page)
    print("Start to crawl NSB BBS at:", datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    for board_url in board_urls:
        print("Start to parse board:", board_url)
        post_data_list = parse_board_url(board_url, post_data_list)
    print("Crawl finished at:", datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    print("Total %d posts are crawled!"%len(post_data_list))  
    save_parse_data(post_data_list)
  

Login successfully!
Start to crawl NSB BBS at: 2017-08-12 22:00:03
Start to parse board: listthreads?forum=7
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=10
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=20
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=30
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=40
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=50
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=60
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=70
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=80
Start to parse page: http://www.nsntr

Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=840
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=850
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=860
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=870
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=880
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=890
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=900
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=910
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=7&offset=920
Start to parse board: listthreads?forum=8
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=8

Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=10
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=20
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=30
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=40
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=50
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=60
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=70
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=80
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=90
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=26&offset=100
Start to parse page: http://w

Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=290
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=300
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=310
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=320
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=330
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=340
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=350
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=360
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=370
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=380
Start to parse page: http://ww

Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1140
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1150
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1160
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1170
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1180
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1190
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1200
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1210
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1220
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=1&offset=1230
Start to parse page:

...

Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1780
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1790
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1800
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1810
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1820
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1830
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1840
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1850
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1860
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=19&offset=1870
Start to p

Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=30
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=40
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=50
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=60
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=70
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=80
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=90
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=100
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=110
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=36&offset=120
Start to parse page: http:/

Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=110
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=120
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=130
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=140
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=150
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=160
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=170
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=180
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=190
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=17&offset=200
Start to parse page:

Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=30&offset=90
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=30&offset=100
Start to parse board: listthreads?forum=33
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=33
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=33&offset=10
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=33&offset=20
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=33&offset=30
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=33&offset=40
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=33&offset=50
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=33&offset=60
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=33&offset

Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=35&offset=270
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=35&offset=280
Start to parse board: listthreads?forum=32
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=32
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=32&offset=10
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=32&offset=20
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=32&offset=30
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=32&offset=40
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=32&offset=50
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=32&offset=60
Start to parse page: http://www.nsntradeunion.com.cn/bbs/mvnforum/listthreads?forum=32&offse